# RSA

Rivest Shamir Adleman (RSA) jest jednym z pierwszych algorytmów asymetrycznych. Został zaprojektowany w 1977 roku i jest używany do dzisiejszego dnia. Zyskał akceptację zarówno NISTu jak i organizacji ISO/IEC oraz RFC. RSA posiada parę kluczy - publiczny oraz prywatny. Publiczny klucz może być znany każdemu i służy on do operacji szyfrowania. Klucz prywatny jest znany tylko i wyłącznie instancji, która klucze generowała. Ta sama instancja jako jedna jedyna ma możliwość odszyfrowania kryptogramów.

RSA umożliwia także tworzenie podpisów cyfrowych (z ang *Digital Signatures*, czyli *DS*). Podpis cyfrowy to dodatkowy blok informacji dołączony do wiadomości, który zapewnia:
1. *Integrity* - integralność wiadomości, czyli potwierdzenie, że nie była ona w żaden sposób modyfikowana.
2. *Authentication* - autentykacje podpisującego, czyli potwierdzenie jego tożsamości.
3. *Non-repudiation* - czyli wysyłający podpisaną wiadomość nie ma możliwości zaprzeczenia faktu, że to on ją podpisał, natomiast otrzymujący wiadomość nie ma możliwości zaprzeczenia faktu, iż to on ją zweryfikował. 

Samo haszowanie wiadomości zapewnia tylko *integirty*, natomiast utworzenie kodu MAC (*Message Authentiaction Code*) zapewnia jedynie *integrity* oraz *authentiaction*.

Tworzenie podpisu cyfrowego z wykorzystaniem RSA wygląda odwrotnie niż komunikacja szyfrowana. To znaczy: podpis tworzony jest z wykorzystaniem klucza prywatnego - a więc tylko instancja generująca klucze może wiadomość podpisać. Weryfikacja odbywa się z wykorzystaniem klucza publicznego - czyli każda instancja, której nasz klucz udostępnimy, może podpis zweryfikować. 

Na początek zaimportujmy niezbędne biblioteki.

In [1]:
# !pip install pycryptodome
import Crypto.Util.number as cu
import hashlib as hl
import math
from Crypto.Hash import SHA256 as sha
from os import urandom
from math import log2 as l2, gcd as GCD, lcm as LCM

**Zadanie 1**

Odpowiedz na poniższe pytania. Rozważ sytuację, w której dwie instancje komunikują się ze sobą, a trzecia próbuje dokonywać nieautoryzowanych zmian w treści wiadomości na drodze jej przesyłu.
1. Dlaczego haszowanie zapewnia tylko integrity? Podpowiedź: czy haszowanie uwzględnia jakikolwiek klucz prywatny?
2. Dlaczego kod MAC nie zapewnia *non-repudiation*? Co (niepożądanego) może dokonać odbierający wiadomość i atakujący komunikację? Podpowiedź: czy kody MAC, w szczególności popularny kod HMAC - który zakłada użycie klucza prywatnego - w jakikolwiek sposób weryfikuje KTO i DO CZEGO go użył? Kto dysponuje, a kto NIE dysponuje kluczem prywatnym HMAC?
3. Dlaczego podpis cyfrowy zapewnia wszystkie te trzy cechy?

Odp.

1. zmiana w wiadomości powoduje zmianę w hashu => integrity. Ale nie wiadomo, kto hash wygenerował
2. Kod MAC wykorzystuje klucz tajny do tworzenia skrótu - stąd zapewnia integrity oraz authentication. Nie zapewnia non-repudation, ponieważ odbiorca może MAC sfałszować (nie da się rozróżnić, która ze stron stworzyła MAC)
3. Podpis cyfrowy może być stworzony tylko przez podpisującego, co można zweryfikować.

## Generowanie kluczy

Algorytm generowania kluczy RSA może zostać przedstawiony w następujący sposób:

1) Znajdź dwie różne i kryptograficznie bezpieczne liczby pierwsze.

2) Oblicz $n = p * q$.

3) Oblicz $f = (p - 1) * (q - 1)$.

4) Znajdź dowolne $e$, takie, że $1 < e < f$ oraz $GCD(f, e) = 1$. GCD to największy wspólny dzielnik. Para $(e, n)$ to jest **klucz publiczny**.

5) Oblicz $d = e^{-1}$ mod $f$. Para $(d, n)$ to **klucz prywatny**, przy czym tajne jest tylko $d$. 

W ten sposób generowane parametry byłyby matematycznie poprawne, lecz kryptograficznie niebezpieczne. Ustalmy więc, że chcemy aby nasz klucz publiczny był odpwowiednio długi. Będzie to długość bitowa parametru $n$, oznaczmy ją jako $nlen = 2048$. Parametr $nlen$ zawsze przyjmuje parzyste wartości. Mając to założenie, musimy (**uwzględniając wszystkie założenia z algorytmu generowania kluczy**) dodatkowo zapewnić, że:

1. $65537 ≤ e < 2^{256}$
2. $LCM(p - 1, q - 1) \geq e * 2^{nlen/2}$
3. $2^{(nlen - 1)//2} < p < 2^{nlen/2}$
4. $2^{(nlen - 1)//2} < q < 2^{nlen/2}$
5. $|p - q| > 2^{(nlen/2) - 100}$

Gdzie LCM oznacza *Least Common Multiple*, czyli najmniejszą wspólną wielokrotność. Funkcję LCM znajdziesz w bibliotece math. Do potęgowania **nie używaj** pythonowej notacji "**", tylko metody pow() - przetestuj obie te metody obliczania potęgi i porównaj wydajność (zadanie opcjonalne). Do obliczania wartości bezwzględnej użyj metody abs() - również standardowa metoda pythona. Resztę niezbędnych metod znajdziesz w bibliotece [Crypto.Util.number](https://pycryptodome.readthedocs.io/en/latest/src/util/util.html) zaimpoertowanej jako cu. Opis powyższych założeń możesz znaleźć w [tym](https://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-56Br2.pdf) dokumencie NIST-owym.

**Zadanie 2**

Uwzględniając wszystko powyższe, napisz metodę/metody generujące wartości $n$, $e$ oraz $d$.

In [2]:
def log2(x):
  return l2(x)

def logger(m):
    print(f'LOGGER::{m}')

def verify_primes(p,q,nlen):
    log_p = log2(p)
    log_q = log2(q)
    log_pq = log2(abs(p-q))
    condp = ((nlen-1)//2 < log_p) and (log_p < nlen/2)
    condq = ((nlen-1)//2 < log_q) and (log_q < nlen/2)
    condpq = log_pq > (nlen/2 - 100)
    return condp and condq and condpq

def verify_pqe(p, q, e, nlen):
    v1 = log2(LCM(p-1,q-1))
    v2 = nlen/2 + log2(e)
    return v1 >= v2

def get_two_primes(nlen):
    prime_l = (nlen)//2
    p = cu.getPrime(prime_l)
    q=p
    while(p==q):
        q = cu.getPrime(prime_l)
    if verify_primes(p,q,nlen):
        return p,q
    return get_two_primes(nlen)

def find_e(f):
    e = 65537
    if(e>=f):
        print("Condition for e impossible to satisfy")
        return None
    gdc = GCD(e,f)
    log2_f = log2(f)
    while(gdc != 1 and log2(e) < min(256, log2_f)):
        e += 1
        gdc = GCD(e,f)
    if(gdc == 1):
        return e
    return None

def generate_key(nlen=2048):
    p,q = get_two_primes(nlen)
    n, f = p*q, (p-1)*(q-1)
    e = find_e(f)
    if not verify_pqe(p,q,e,nlen):
        return generate_key(nlen)
    d = cu.inverse(e,f)
    public_key = (e,n)
    private_key = (d,n)
    logger('private_key, public_key generated')
    return public_key, private_key 
    

## Naiwne szyfrowanie i deszyfrowanie


Naiwny algorytm szyfrowania wiadomości **M** z wykorzystaniem RSA:

1) Zakoduj $M$ jako liczbę.

2) Oblicz: $C = M^e$ mod $n$.

Naiwny algorytm deszyfrowania kryptogramu **C** z wykorzystaniem RSA:

1) $M = C^d$ mod $n$.

2) Zdekoduj wiadomość $M$ do jej pierwotnej postaci (np. stringa). 


**Zadanie 3**

Napisz metody szyfrujące i deszyfrujące wiadomość $M$ zgodnie z powyższym algorytmem. Zaszyfruj wiadomość, zdeszyfruj i wypisz oryginalny tekst na ekranie. Odpowiedz na pytanie: jaki warunek musi spełniać liczbowa reprezentacja wiadomości $M$, aby można ją było poprawnie zaszyfrować i zdeszyfrować?

In [3]:
def convert_to_int(msg):
    if(type(msg)==str):
        msg = [ord(i) for i in msg]
    l = len(msg)
    num = 0
    base = 2**8 
    for i in range(l):
        num += msg[l-i-1]*(base**i)
    return num
    # return convert_array_to_int(msg)

def convert_from_int(num):
    A = []
    base = 2**8
    while(num > 0):
        A.append(num%base)
        num = (num-A[-1])//base
    A.reverse()
    A = [chr(i) for i in A]
    A = ''.join(A)
    return A

def RSA_simple_encode(msg, public_k):
    logger(f'encoding...')
    M = convert_to_int(msg)
    e,n = public_k
    c = M**e % n
    return c
def RSA_simple_decode(c, private_k):
    logger(f'decoding...')
    d, n = private_k
    M = pow(c, d, n) # c**d % n
    return convert_from_int(M)


In [4]:
# test conversion

M = "Ala ma kota"
print([hex(ord(i)) for i in M])
msg = convert_to_int(M)
print(hex(msg))
x = convert_from_int(msg)
print(x)

['0x41', '0x6c', '0x61', '0x20', '0x6d', '0x61', '0x20', '0x6b', '0x6f', '0x74', '0x61']
0x416c61206d61206b6f7461
Ala ma kota


In [5]:
# test simple RSA

M = "Ala ma kota"
print(M)
public_key, private_key = generate_key()
c = RSA_simple_encode(M, public_key)
print(f'encoded: {c}')

Ala ma kota
LOGGER::private_key, public_key generated
LOGGER::encoding...
encoded: 8885802931972636400819081967051314742815917964232843002726056756618448726682420220398749404438511944962846571541943357713112153471423411655626228679559459891168893909086163838306599932745000401337055166602332536390175449608388296960528502445836811397021720401572897606741605390122866132985165680703762633368035859881648539508527488543342337777427304482692079487675798711893197750227820084075496564296555258233128884749809125214658611458805896304159119637960487397398758157521685004551605147780756909168784343345524374433814398977787540950556076990930635692316592953326582689145376317660570993245454738623732794223545


In [6]:
e = RSA_simple_decode(c, private_key)
print(e)

LOGGER::decoding...
Ala ma kota


## Naiwny schemat podpisu cyfrowego

Naiwna metoda tworzenia podpisu z wiadmości $M$:

1) Oblicz $h = H(M)$. H to uzgodniona funkcja skrótu, niech to będzie SHA-256.

2) Zakoduj $h$ jako liczbę.

3) Oblicz $SIG = h^d$ mod $n$.

4) Wyślij parę $(M, SIG)$ weryfikującemu.

Naiwna metoda weryfikacji podpisu $(M, SIG)$:

1) Oblicz $h = H(M)$. H to uzgodniona funkcja skrótu, niech to będzie SHA-256.

2) Zakoduj $h$ jako liczbę.

3) Oblicz $VER = SIG^e$ mod $n$.

4) Jeżeli $VER = h$, weryfikacja przebiegła pomyślnie, a w przeciwnym razie niepomyślnie.

**Zadanie 4**

Zaimplementuj naiwną metodę tworzenia i weryfikowania podpisu cyfrowego RSA.

In [7]:
def convert_without_overflow(msg, MAX_BYTES = 4294967296):
    success = False
    number_of_bytes = 32
    new_msg = msg
    while(number_of_bytes<MAX_BYTES and not success):
        try:
            new_msg = msg.to_bytes(number_of_bytes, byteorder='big')
            success = True
        except OverflowError:
            number_of_bytes *= 2
    if(success is False):
        print('CONVERSION FAILED!!!')
        return None
    return new_msg


def SHA256(msg):
    if(type(msg)==str):
        msg = msg.encode('ascii')
    elif(type(msg)==int):
        msg = convert_without_overflow(msg)
    h = sha.new()
    h.update(msg)
    return int(h.hexdigest(), 16)

#test
x = SHA256(pow(2, 512))
# x = SHA256('ala ma kota')
print(x)

73760069341810921996523761465530118473534114516935146510527818014149546661271


In [8]:
def RSA_simple_sign(msg, private_k, H = SHA256):
    h = H(msg)
    d, n = private_k
    sig = pow(h, d, n) # h**d % n
    return sig

def RSA_simple_verify(M, sig, public_k, H = SHA256):
    e, n = public_k
    h = H(M)
    ver = pow(sig, e, n) # sig**e % n
    return ver==h

In [9]:
# test RSA signature

M = "Ala ma kota"

public_key, private_key = generate_key()
sig = RSA_simple_sign(M, private_key)
print(sig)
verification = RSA_simple_verify(M, sig, public_key)
print(verification)


LOGGER::private_key, public_key generated
13898764422170759514417998296207789090831955948724614566044983741176135293375330496643004340549123030316690683756694367469621487863338348644371353500963605230121733652862833484900087836798579004567693736549141879402658793764652340503212400666497120783837632720171953333259803485805009458157938913459208288353508102694750041080965302327525118439556499023742281295130942263939366752627139663244851804243396046342501613386991533633221110373665671744965862634786902767616755394756423275876940119991320930818848397130225200191653867008483326864351035941205834144163042124444485326966452412694184011159313940553066726095861
True


## MGF 1

W dalszej części laboratoriów będziemy potrzebowali generować maskę. Jedynym zatwierdzonym algorytmem który do tego służy jest *Mask Generation Function 1*, opisany w [RFC 8017](https://www.rfc-editor.org/rfc/rfc8017). Jest on stosunkowo prosty. 

Parametry wejściowe:

1) M - bajty wiadomości.

2) len - pożądana długość zwórconej maski w bajtach.

3) H - wybrana funkcja skrótu, zwracająca $n$ bitowy skrót. Niech to będzie SHA-256. Dla wygody przyjmijmy też, że $hlen = n / 8$ oznacza liczbę bajtów zwracaną przez naszą funkcję skrótu.

Wyjściem funkcji są bajty tworzące maskę.

Algorytm MGF-1:

1) Dla 32-biotwego integera $i = 0, ..., ⌈ \frac{len}{hlen}⌉ - 1$ wykonuj kroki 2 i 3.

2) Oblicz tmp = H(M || i). Znak || to konkatenacja i chodzi tu o bajty wiadomości M oraz reprezentację w bajtach 32-bitowego itegera $i$.

3) Oblicz output = output || tmp.

4) Zwróc $len$ wiodących bajtów zmiennej output.

**Zadanie 5**

Zaprogramuj i przetestuj dla dowolnych wartości funkcję MGF1.

In [10]:
def concatenate(x, y, base=32):
    if(type(x) is not int):
        int.from_bytes(x, byteorder='big')
    res = (x<<base) | (y)
    return res

def get_x_bytes(num, x, base=2):
    A = []
    while(num > 0):
        A.append(num%base)
        num = (num-A[-1])//base
    A.reverse()
    A = A[:x]
    num = 0
    l = len(A)
    for i in range(l):
        num += A[l-i-1]*pow(base,i)
    return num

def mgf_1(M, l, H = SHA256):
    h_l = 256//8
    output = 0
    for i in range(math.ceil(l/h_l)):
        M1 = concatenate(M, i, base=32)
        tmp = H(M1)
        output = concatenate(output, tmp)
    return get_x_bytes(output, l)

In [11]:
x, y = 0x45ef11, 0xaa3276
z = concatenate(x,y,base=32)
print(hex(z))
q = get_x_bytes(z,10) #22f
print(hex(q))

0x45ef1100aa3276
0x22f


In [12]:
# test mgf_1

def generate_seed(l):
    '''WYgeneruj losową tablicę bajtów o długości l'''
    x = urandom(l)
    x = [i for i in x]
    return x

l = 256
seed = generate_seed(256)
seed = convert_to_int(seed)
# print(seed)
x = mgf_1(seed, 64)
print(f'mgf_1 result = {x}')

mgf_1 result = 14618399969025389310


## OAEP

Nasz schemat ma na ten moment jedną sporą wadę, mianowicie rozmiar szyfrowanej wiadomości może być zbyt mały, czyniąc algorytm mniej bezpiecznym. Aby tego uniknąć, używamy algorytmu paddingu opisanego w [RFC 8017](https://www.rfc-editor.org/rfc/rfc8017#section-8), który zwie się *Optimal Assymetric Encryption Padding*.

### OAEP encoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $k$ - długość liczby $n$ wyrażona w bajtach.

3) $mlen$ - długość wiadomości wyrażona w bajtach.

4) $M$ - bajty wiadomości.

5) $mgf1$ - Mask Generation Function 1.

Algorytm:

1) Jeżeli $mlen > k - 2*hlen - 2$ zwróc błąd.

2) Oblicz: $lHash = H("")$.

3) Wygeneruj tablicę bajtów $PS$ składającą się z $k - mlen - 2*hlen - 2$ bajtów o wartości 0x00. Rozmiar $PS$ może wynosić 0.

4) Oblicz: $DB = lHash || PS || 0x01 || M$. Długość $DB$ powinna wynosić $k - hlen - 1$ bajtów.

5) Wygeneruj losową tablicę bajtów $seed$ o rozmiarze $hlen$.

6) Oblicz: $dbMask = mgf1(seed, k - hlen - 1)$.

7) Oblicz: $maskedDB = DB ⊕ dbMask$.

8) Oblicz: $seedMask = mgf1(maskedDB, hlen)$.

9) Oblicz: $maskedSeed = seed ⊕ seedMask$.

10) Oblicz: $EM = 0x00 || maskedSeed || maskedDB$. Długość $EM$ powinna wynosić $k$.

11) Zwróc $EM$.

### OAEP decoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $k$ - rozmiar EM wyrażony w bajtach.

3) $mgf1$ - Mask Generation Function 1.

4) $EM$ - bajty zakodowanej wiadomości.

Algorytm:

1) Rozpakuj tablicę bajtów $EM$. Jej pierwszy bajt (najbardziej znaczący) przypisz do $Y$. Kolejne $hlen$ bajtów przypisz do $maskedSeed$, resztę do $maskedDB$. Czyli $EM = Y || maskedSeed || maskedDB$.

2) Oblicz: $lHash = H("")$.

3) Oblicz: $seedMask = mgf1(maskedDB, hlen)$.

4) Oblicz: $seed = maskedSeed ⊕ seedMask$.

5) Oblicz: $dbMask = mgf1(seed, k - hlen - 1)$.

6) Oblicz: $DB = maskedDB ⊕ dbMask$.

7) Rozpkauj tablicę bakjtów $DB$. Pierwsze (najbardziej znaczące) $hlen$ bajtów przypisz do zmiennej $lHash'$. Następne $k - mlen - 2*hlen - 2$ bajtów do PS. Kolejny pojedynczy bajt powinien wynosić 0x01, jeżeli jest inaczej zwróć błąd i **zakończ działanie**. Resztę bajtów przypsiz do zmiennej $M$. Czyli: $DB = lHash' || PS || 0x01 || M$. 

8) Jeżeli $Y \neq 0x00$ zwróć błąd i **zakończ działanie**.

9) Jeżeli $lHash \neq lHash'$ zwróć błąd i **zakończ działanie**.

10) Zwróc $M$.

**Zadanie 6**

Zaproogramuj kodowanie i dekodowanie OAEP. Zmodyfikuj algorytm szyfrowania RSA, tak, aby przed zaszyfrowaniem wiadomość była paddingowana. Zmodyfikuj algorytm deszyfrowania tak, aby po zdeszyfrowaniu konieczne było wywołanie metody dekodowania OAEP w celu odzyskania wiadomości.

In [15]:
M = "Ala ma kota"

def throw(msg):
    print(f'ERROR:{msg}')

def gen_PS(byte_0_num):
    '''Wygeneruj tablicę bajtów $PS$ składającą się z $k - mlen - 2*hlen - 2$ 
    bajtów o wartości 0x00. Rozmiar $PS$ może wynosić 0.'''
    return [0x00 for i in range(byte_0_num)]

def generate_seed(l):
    '''WYgeneruj losową tablicę bajtów o długości l'''
    x = urandom(l)
    x = [i for i in x]
    return x

def combine_bytes(bytes):
    l = len(bytes)
    num = 0
    base = 2**8 
    for i in range(l):
        num += bytes[l-i-1]*(base**i)
    return num

def unpack(num, lengths):
    A = []
    base = 2
    while(num > 0):
        A.append(num%base)
        num = (num-A[-1])//base
    A.reverse()
    # get Y
    result = []
    idx=0
    for l in lengths:
        result.append(A[idx:(idx+l)])
        l = idx+l
    return [combine_bytes(i) for i in result]

def oaep_encode(k, mlen, M, H = SHA256, hlen = 256, mgf = mgf_1):
    if(mlen > (k-1*hlen-2)):
        throw('error 1')
        return None
    lHash = H('')
    PS = gen_PS(k - mlen - 2*hlen - 2)
    DB = concatenate(concatenate(lHash, PS), concatenate(0x01,M))
    # print(f'lHash, PS, 0x01, M = {lHash}:{PS}:01:{M}')
    # print(f'DB = {DB}')
    if(len(DB) != (k - hlen - 1)):
        throw('error 2')
        return None
    seed = generate_seed(hlen)
    dbMask = mgf(seed, k-hlen-1)
    maskedDB = DB ^ dbMask
    seedMask = mgf(maskedDB, hlen)
    maskedSeed = seed ^ seedMask
    EM = concatenate(concatenate(0x00, maskedSeed), maskedDB)
    # print(f'0x00, maskedSeed, maskedDB = 00:{maskedSeed}:{maskedDB}')
    # print(f'EM = {EM}')
    return EM

def oaep_decode(k, mlen, EM, H = SHA256, hlen = 256, mgf = mgf_1):
    print(f'len EM should be {1 + hlen + (k - hlen - 1)}')
    Y, maskedSeed, maskedDB = unpack(EM, [1, hlen, (k - hlen - 1)])
    lHash = H('')
    seedMask = mgf(maskedDB, hlen)
    seed = maskedSeed ^ seedMask
    dbMask = mgf(seed, k - hlen - 1)
    DB = maskedDB ^ dbMask
    print(f'len DB should be {hlen+ k-mlen-2*hlen-2+ 1+ mlen}')
    lHash2, PS, M = unpack(DB, [hlen, k-mlen-2*hlen-2, 1, mlen])
    if(Y != 0x00 or lHash != lHash2):
        return None
    return M
    

In [18]:
# RSA

def RSA_oaep_encode(msg, public_k):
    logger(f'encoding...')
    M = convert_to_int(msg)
    e,n = public_k
    M = oaep_encode(n.bit_length()//8, M.bit_length()//8, M)
    c = M**e % n
    return c

def RSA_oaep_decode(c, private_k):
    logger(f'decoding...')
    d, n = private_k
    M = pow(c, d, n) # c**d % n
    M = oaep_decode(n.bit_length()//8, M.bit_length()//8, M)
    return convert_from_int(M)

In [19]:
M = "Ala ma kota"
print(M)
public_key, private_key = generate_key()
c = RSA_oaep_encode(M, public_key)
print(f'encoded: {c}')

Ala ma kota
LOGGER::private_key, public_key generated
LOGGER::encoding...
ERROR:error 1


TypeError: unsupported operand type(s) for ** or pow(): 'NoneType' and 'int'

In [ ]:
e = RSA_oaep_decode(c, private_key)
print(e)

## EMSA - PSS

Utworzenie bezpiecznej sygnatury RSA wymaga zastowania algorytmu *Encoding Method for Signature with Appendix - Probabilistic Signature Scheme* .

### EMSA encoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $slen$ - długość soli w bajtach, powinna być równa $hlen$.

3) $M$ - bajty wiadomości do podpisania.

4) $mgf1$ - Mask Generation Function 1.

5) $emBits$ - pożądana długość sygnatury w bitach. Jest to najczęściej długość bitowa liczby modulus $n$ pomniejszona o jeden, czyli w naszym przypadku 2047.

6) $emlen$ - długość sygnatury w bajtach, równa długości parametru $n$ wyrażonego w bajtach.


Algorytm:

1) Oblicz: $mHash = H(M)$.

2) Jeżeli $emlen < hlen + slen + 2$ **zakończ i zwróć błąd**.

3) Wygeneruj tablicę losowych bajtów $salt$ o długości $slen$.

4) Oblicz: $M' = 0x00 00 00 00 00 00 00 00 || mHash || salt$. Długość $M'$ to $8 + hlen + slen$.

5) Oblicz: $mHash' = H(M')$.

6) Wygeneruj tablicę $PS$ składającą się z bajtów 0x00 o długości $emlen - slen - hlen - 2$.

7) Oblicz: $DB = PS || 0x01 || salt$. Długość $DB$ powinna wynosić $emlen - hlen - 1$ bajtów.

8) Oblicz: $dbMask = mgf1(mHash', emlen - hlen - 1)$.

9) Oblicz: $maskedDB = DB ⊕ dbMask$.

10) Ustaw $8 * emlen - emBits$ **najbardziej znaczących** bitów $maskedDB$ na wartości 0.

11) Oblicz: $EM = maskedDB || mHash' || 0xbc$.

12) Zwróć $EM$.


## EMSA decoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $slen$ - długość soli w bajtach, powinna być równa $hlen$.

3) $EM$ - sygnatura wiadomości $M$.

4) $M$ - bajty wiadomości do weryfikacji.

4) $mgf1$ - Mask Generation Function 1.

5) $emBits$ - długość sygnatury w bitach. Jest to najczęściej długość bitowa liczby modulus $n$ pomniejszona o jeden, czyli w naszym przypadku 2047.

6) $emlen$ - długość sygnatury w bajtach, równa długości parametru $n$ wyrażonego w bajtach.

Algorytm:

1) Oblicz: $mHash = H(M)$.

2) Jeżeli $emlen < hlen + slen + 2$ **zakończ i zwróć błąd weryfikacji**.

3) Jeżeli ostatni bajt (najmniej znaczący) $EM$ nie ma wartości 0xbc **zakończ i zwróć błąd weryfikacji**.

4) Podstaw $emlen - hlen - 1$ najbardziej znaczących bajtów do $maskedDB$ oraz kolejne $hlen$ bajtów do $mHash'$.

5) Jeżeli $8 * emlen - emBits$ najbardziej znaczących bitów $maskedDB$ nie ma wartości 0, **zakończ i zwróć błąd weryfikacji**.

6) Oblicz: $dbMask = mgf1(mHash', emlen - hlen - 1)$.

7) Oblicz: $DB = maskedDB ⊕ dbMask$.

8) Ustaw $8 * emlen - emBits$ najbardziej znaczących bitów $DB$ na 0.

9) Jeżeli $emlen - hlen - slen - 2$ najbardziej znaczących bajtów $DB$ nie posiada wartości 0x00 lub gdy bajt na pozycji $emlen - hlen - slen - 1$ (licząc od najbardziej znaczącego) nie posiada wartości 0x01 **zakończ i zwróć błąd weryfikacji**.

10) Przypisz do zmiennej $salt$ dokładnie $slen$ najmniej znaczących bajtów $DB$.

11) Oblicz: $M' = 0x00 00 00 00 00 00 00 00 || mHash || salt$. Długość $M'$ to $8 + hlen + slen$.

12) Oblicz $mHash'' = H(M')$.

13) Jeżeli $mHash' \neq mHash''$ **zakończ i zwróć błąd weryfikacji**, w przeciwnym razie **weryfikacja powiodła się**.


**Zadanie 7**

Zaprogramuj kodowanie i dekodowanie EMSA, a następnie popraw algorytmy tworzenia i weryfikacji podpisu cyfrowego RSA. Tworzenie podpisu powinno wyglądać tak, że wiadomość najpierw jest kodowana z wykorzystaniem EMSA, a później tworzony jest popdis z wykorzystaniem klucza prywatnego. Dekodowanie powinno wyglądać tak, że najpierw używany jest klucz publiczny do odtworzenia podpisu EMSA, a następnie wykorzystywane jest dekodowanie EMSA w celu weryfikacji.

In [78]:
m = "Ala ma podpis"

def generate_salt(slen):
    pass

def convert_int_to_arr(num):
    A = []
    base = 2**8
    while(num > 0):
        A.append(num%base)
        num = (num-A[-1])//base
    A.reverse()
    return A

def set_x_bytes(num, x, base=2):
    A = []
    while(num > 0):
        A.append(num%base)
        num = (num-A[-1])//base
    A.reverse()
    # A = A[:x]
    for i in range(x):
        A[i] = 0x00
    num = 0
    l = len(A)
    for i in range(l):
        num += A[l-i-1]*(base**i)
    return num

def emsa_encoding(M, emBits, emlen, slen, H = SHA256, hlen = 256, mgf = mgf_1):
    mHash = H(M)
    if(emlen < hlen+slen+2):
        throw(f'error emsa encoding')
        return None
    salt = generate_salt(hlen)
    M2 = concatenate(0x0000000000000000, concatenate(mHash,salt))
    assert len(M2) == 8+hlen+slen
    mHash2 = H(M2)
    PS = gen_PS(emlen-slen-hlen-2)
    DB = concatenate(PS, concatenate(0x01,salt))
    assert len(DB) == emlen-hlen-1
    dbMask = mgf(mHash2, emlen-hlen-1)
    maskedDB = DB ^ dbMask
    maskedDB = set_x_bytes(maskedDB, 8*emlen-emBits)
    EM = concatenate(maskedDB, concatenate(mHash2, 0xbc))
    return EM

def emsa_decoding(M, EM, emBits, emlen, slen, H = SHA256, hlen = 256, mgf = mgf_1):
    mHash = H(M)
    if(emlen < hlen+slen+2):
        throw(f'error emsa decoding')
        return None
    em_bytes = convert_int_to_arr(EM)[-1]
    if(0xbc != em_bytes):
        throw(f'error emsa: last bit check failed')
        return None
    # instructions unclear
    byte_num = 8*emlen-emBits
    em_bytes1 = get_x_bytes(EM, byte_num+hlen)
    em_bytes2 = get_x_bytes(em_bytes1, byte_num)
    set_x_bytes(maskedDB, em_bytes1)
    set_x_bytes(mHash2, em_bytes1)
    if(get_x_bytes(maskedDB) != 0x00):
        throw(f'error emsa: maskedDB check failed')
        return None
    dbMask = mgf(mHash2, emlen-hlen-1)
    DB = maskedDB ^ dbMask
    set_x_bytes(DB, byte_num)
    #step 9: verification
    #step 10

In [ ]:
def RSA_emsa_encode(msg, public_k):
    msg_emsa = emsa_encoding()

def RSA_simple_encode(msg, public_k):
    M = convert_to_int(msg)
    nlen = 2048 # ????
    # public_k, private_k = generate_key(nlen)
    e,n = public_k
    c = M**e % n
    return c

def RSA_simple_decode(c, private_k):
    d, n = private_k
    M = c**d % n
    return convert_from_int(M)


## RSA - KEM

RSA - KEM, czyli z ang. *Key Encapsulation Mechanism* to prosty algorytm dedykowany do wymiany klucza symetrycznego. Obie strony dysponują uzgodnioną funkcją skótu H. Instancja, która chce **otrzymać** tajny klucz do komunikacji symetrycznej generuje klucze RSA i udostępnia swój klucz publiczny. Instancja, która chce wygenerować tajny klucz do komunikacji symetrycznej dysponuje kluczem publicznym instancji, która chce go otrzymać.

Instancja generująca klucz symetryczny:

1) Znajdź losową liczbę $RAND$ spełniającą warunki OAEP.

2) Oblicz: $KEY = H(RAND)$. Jeżeli trzeba, przytnij $KEY$ do odpowiedniej długości.

3) Oblicz: $CIPHERED\_KEY = RSA\_OAEP\_ENCODING(KEY, (e, n))$.

4) Wyślij $CIPHERED\_KEY$.

Instancja otrzymująca zaszyfrowany klucz symetryczny:

1) Oblicz: $KEY = RSA\_OAEP\_DECODING(CIPHERED\_KEY, (d, n))$

2) Jeżeli trzeba przytnij $KEY$ do odpowiedniej długości.

Np. AES występuje w wersji 128b, 192b i 256b. Jeżeli jako H przyjmiemy więc SHA-256, nie trzeba przycinać klucza dla algorytmu AES-256. W przeciwnym razie należy klucz odpowiednio przyciąć (z lewej lub prawej, byle obie strony tak samo) i to ta wartość staje się kluczem symetrycznym.

**Zadanie 8**

Zasymuluj takową wymianę (bez przycinania klucza).

In [ ]:
# Your code here